In [1]:
#%%capture

# run manually in terminal in the future
"""
# Update package list and install Python 3.9
!sudo apt-get update
!sudo apt-get install -y python3.9

# Set up update-alternatives to point to the new Python version
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 2

# Select the new Python version as default
!sudo update-alternatives --config python3

# Verify the Python version
!python3 --version

# Upgrade pip and install necessary packages
!python3 -m pip install --upgrade pip
!python3 -m pip install jupyter
"""

'\n# Update package list and install Python 3.9\n!sudo apt-get update\n!sudo apt-get install -y python3.9\n\n# Set up update-alternatives to point to the new Python version\n!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1\n!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 2\n\n# Select the new Python version as default\n!sudo update-alternatives --config python3\n\n# Verify the Python version\n!python3 --version\n\n# Upgrade pip and install necessary packages\n!python3 -m pip install --upgrade pip\n!python3 -m pip install jupyter\n'

In [2]:
# Verify the installation
!python --version
!python3 --version

Python 3.9.5
Python 3.9.5


In [3]:
"""
!git clone https://github.com/nlp-with-transformers/notebooks.git
%cd notebooks
#from install import *
#install_requirements(is_chapter6=True)
"""
%cd notebooks

/dli/notebooks


/usr/local/lib/python3.9/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
#%%capture
#"""
!pip install transformers==4.41.2
!pip install datasets==2.20.0

!pip install pyarrow==16.0
!pip install requests==2.32.3

!pip install torch==2.3.0 torchvision==0.18.0 torchaudio==2.3.0

!pip install importlib-metadata

!pip install accelerate -U

!pip install psutil==6.0.0


# Specific to Nvidia Instance
!pip install matplotlib==3.9.1
!pip install --upgrade cython
!pip install --upgrade --force-reinstall ipython
!pip install pickleshare
#"""

In [5]:
from utils import *
setup_chapter()

Using transformers v4.41.2
Using datasets v2.20.0


In [6]:
#%%capture
# Verifying packages installed are now up to date
!pip show pyarrow requests transformers datasets torch torchaudio importlib-metadata

Name: pyarrow
Version: 16.0.0
Summary: Python library for Apache Arrow
Home-page: https://arrow.apache.org/
Author: 
Author-email: 
License: Apache License, Version 2.0
Location: /usr/local/lib/python3.9/dist-packages
Requires: numpy
Required-by: datasets
---
Name: requests
Version: 2.32.3
Summary: Python HTTP for Humans.
Home-page: https://requests.readthedocs.io
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache-2.0
Location: /usr/local/lib/python3.9/dist-packages
Requires: certifi, charset-normalizer, idna, urllib3
Required-by: datasets, huggingface-hub, jupyterlab_server, transformers
---
Name: transformers
Version: 4.41.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 

In [7]:
!nvidia-smi

Tue Jul  9 14:04:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          On  | 00000001:00:00.0 Off |                    0 |
| N/A   39C    P0              53W / 300W |      7MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
!cat /proc/cpuinfo

# Training Transformers from Scratch

In [9]:
# First comparing gpt and gpt-2
from transformers import pipeline, set_seed

generation_gpt = pipeline("text-generation", model="openai-gpt")
generation_gpt2 = pipeline("text-generation", model="gpt2")

config.json:   0%|          | 0.00/656 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/479M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/816k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [10]:
# calculates and shows model size
def model_size(model):
    return sum(t.numel() for t in model.parameters())

print(f"GPT  size: {model_size(generation_gpt.model)/1000**2:.1f}M parameters")
print(f"GPT2 size: {model_size(generation_gpt2.model)/1000**2:.1f}M parameters")

GPT  size: 116.5M parameters
GPT2 size: 124.4M parameters


In [11]:
set_seed(1)

In [12]:
# The function below converts the output from the transformers which are in the form of a list to a continuous text for printing.
                # function below is to show effects of skewed dataset, since bookcorpus which GPT trained on was very skewed towards romance genre, we can see how different
                # their respective output themes are
def enum_pipeline_ouputs(pipe, prompt, num_return_sequences):
    out = pipe(prompt, num_return_sequences=num_return_sequences,
               clean_up_tokenization_spaces=True)
    return "\n".join(f"{i+1}." + s["generated_text"] for i, s in enumerate(out))

prompt = "\nWhen they came back"
print("GPT completions:\n" + enum_pipeline_ouputs(generation_gpt, prompt, 3))
print("")
print("GPT-2 completions:\n" + enum_pipeline_ouputs(generation_gpt2, prompt, 3))

GPT completions:
1.
When they came back, she 'd want to know what he was going to do.
 " it 'll teach your brother a lesson, " he said, and then he looked pointedly
at my arm and the blood he 'd poured over my skin. "
2.
When they came back, a little boy with a black nose and brown eyes who had
started hanging around the neighborhood on special occasions and who lived off
of the property. he wore a football jersey with the letters " the black diamond
" written on it.
3.
When they came back, and they were still there. "
 jake nodded.
 i shrugged and sat back down. " so then, they did come back, " i said,
thinking. " but... "
 " what's her name?

GPT-2 completions:
1.
When they came back the body was still on fire, and there were a lot of people
who had died. The smell of diesel had been getting worse. No one was doing well.
The streets were deserted, with no water and no running water
2.
When they came back to the door, they came back with the fire extinguisher.

The man, who was carryi

---
## Building a Custom Code dataset

This will be achived using Google Cloud's BigQuery
- nevermind. Managed to get google cloud bigquery working. However, seems to be quota issues with free tier account. Not bothered with doing this the hard way so easy way it is below.

In [14]:
TOKENIZERS_PARALLELISM=False

In [13]:
!git clone https://huggingface.co/datasets/transformersbook/codeparrot

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Cloning into 'codeparrot'...
remote: Enumerating objects: 196, done.
remote: Total 196 (delta 0), reused 0 (delta 0), pack-reused 196 (from 1)
Receiving objects: 100% (196/196), 28.24 KiB | 14.12 MiB/s, done.
Resolving deltas: 100% (4/4), done.
